Step one: Reformatting corpus data

The data that I'll be using to train the taggers is in .conllu format, and the NLTK taggers require a different format, so the first step will be to change the training data into a usable format for the taggers. 





In [6]:
from conllu import parse

file = open('../../UD_Indonesian-GSD/id_gsd-ud-train.conllu', 'r')
f = file.read()
file.close()

print(f[:1393])


# sent_id = train-s1
# text = Sembungan adalah sebuah desa yang terletak di kecamatan Kejajar, kabupaten Wonosobo, Jawa Tengah, Indonesia.
1	Sembungan	sembungan	PROPN	X--	_	4	nsubj	_	MorphInd=^sembungan<x>_X--$
2	adalah	adalah	AUX	O--	_	4	cop	_	MorphInd=^adalah<o>_O--$
3	sebuah	sebuah	DET	B--	PronType=Ind	4	det	_	MorphInd=^sebuah<b>_B--$
4	desa	desa	NOUN	NSD	Number=Sing	0	root	_	MorphInd=^desa<n>_NSD$
5	yang	yang	PRON	S--	PronType=Rel	6	nsubj:pass	_	MorphInd=^yang<s>_S--$
6	terletak	terletak	VERB	VSP	Number=Sing|Voice=Pass	4	acl	_	MorphInd=^ter+letak<n>_VSP$
7	di	di	ADP	R--	_	8	case	_	MorphInd=^di<r>_R--$
8	kecamatan	kecamatan	NOUN	NSD	Number=Sing	6	obl	_	MorphInd=^ke+camat<n>+an_NSD$
9	Kejajar	kejajar	PROPN	X--	_	8	flat	_	SpaceAfter=No|MorphInd=^kejajar<x>_X--$
10	,	,	PUNCT	Z--	_	8	punct	_	MorphInd=^,<z>_Z--$
11	kabupaten	kabupaten	NOUN	NSD	Number=Sing	8	appos	_	MorphInd=^kabupaten<n>_NSD$
12	Wonosobo	wonosobo	PROPN	X--	_	11	flat	_	SpaceAfter=No|MorphInd=^wonosobo<x>_X--$
13	,	,	PUNCT


Fortunately, there is already a tool for this: Emil Stenstrom's conllu parser (https://github.com/EmilStenstrom/conllu) will turn the .conllu data into a nested dictionary

In [7]:
sentences = parse(f)
print(sentences[:5])

[TokenList<Sembungan, adalah, sebuah, desa, yang, terletak, di, kecamatan, Kejajar, ,, kabupaten, Wonosobo, ,, Jawa, Tengah, ,, Indonesia, .>, TokenList<Sebuah, serangan, pengayauan, biasanya, terjadi, di, ladang, atau, dengan, membakar, sebuah, rumah, dan, memenggal, semua, penghuninya, ketika, mereka, melarikan, diri, .>, TokenList<Perkembangan, ini, diikuti, oleh, helm, Brodie, yang, dipakai, tentara, Imperium, Britania, dan, AS, ,, dan, pada, tahun, 1916, oleh, Stahlhelm, Jerman, dengan, perbaikan, desain, yang, masih, dipakai, sampai, sekarang, .>, TokenList<Dari, jarak, dekat, ,, dua, kapal, perusak, ,, Sterett, dan, O, ', Bannon, menembakkan, beberapa, kali, salvo, ke, bangunan, atas, kapal, Hiei, ., Sebelum, melarikan, diri, ke, dalam, kegelapan, ,, keduanya, mungkin, sempat, menyarangkan, satu, atau, dua, buah, torpedo, ke, bagian, lambung, kapal, Hiei, hingga, makin, memperparah, kerusakan, .>, TokenList<Angka, harapan, hidup, adalah, 73, ,, 4, tahun, ,, yakni, di, bawah, rer

In [8]:
print(sentences[0][0])

Token([('id', 1), ('form', 'Sembungan'), ('lemma', 'sembungan'), ('upos', 'PROPN'), ('xpos', 'X--'), ('feats', None), ('head', 4), ('deprel', 'nsubj'), ('deps', None), ('misc', OrderedDict([('MorphInd', '^sembungan<x>_X--$')]))])


In [9]:
training_data = []

for sentence in sentences:
    tagged_sentence = []
    for token in sentence:
        tagged_sentence.append((token['form'],token['upos']))
    training_data.append(tagged_sentence)
    
print(training_data[:5])

[[('Sembungan', 'PROPN'), ('adalah', 'AUX'), ('sebuah', 'DET'), ('desa', 'NOUN'), ('yang', 'PRON'), ('terletak', 'VERB'), ('di', 'ADP'), ('kecamatan', 'NOUN'), ('Kejajar', 'PROPN'), (',', 'PUNCT'), ('kabupaten', 'NOUN'), ('Wonosobo', 'PROPN'), (',', 'PUNCT'), ('Jawa', 'PROPN'), ('Tengah', 'PROPN'), (',', 'PUNCT'), ('Indonesia', 'PROPN'), ('.', 'PUNCT')], [('Sebuah', 'DET'), ('serangan', 'NOUN'), ('pengayauan', 'NOUN'), ('biasanya', 'ADV'), ('terjadi', 'VERB'), ('di', 'ADP'), ('ladang', 'NOUN'), ('atau', 'CCONJ'), ('dengan', 'ADP'), ('membakar', 'VERB'), ('sebuah', 'DET'), ('rumah', 'NOUN'), ('dan', 'CCONJ'), ('memenggal', 'VERB'), ('semua', 'DET'), ('penghuninya', 'NOUN'), ('ketika', 'SCONJ'), ('mereka', 'PRON'), ('melarikan', 'VERB'), ('diri', 'NOUN'), ('.', 'PUNCT')], [('Perkembangan', 'NOUN'), ('ini', 'DET'), ('diikuti', 'VERB'), ('oleh', 'ADP'), ('helm', 'NOUN'), ('Brodie', 'PROPN'), ('yang', 'PRON'), ('dipakai', 'VERB'), ('tentara', 'NOUN'), ('Imperium', 'PROPN'), ('Britania', 'PR

In [10]:
test_file = open('../../UD_Indonesian-GSD/id_gsd-ud-test.conllu', 'r')
test_f = test_file.read()
test_file.close()

test_sentences = parse(test_f)

test_data = []

for sentence in test_sentences:
    tagged_sentence = []
    for token in sentence:
        tagged_sentence.append((token['form'],token['upos']))
    test_data.append(tagged_sentence)

In [11]:
import nltk

default_tagger = nltk.DefaultTagger('NOUN')

unigram_tagger = nltk.UnigramTagger(training_data, backoff=default_tagger)
unigram_tagger_accuracy = unigram_tagger.evaluate(test_data)

print('Unigram tagger accuracy: ' + str(unigram_tagger_accuracy))

Unigram tagger accuracy: 0.8404074702886248


In [12]:
bigram_tagger = nltk.BigramTagger(training_data, backoff=unigram_tagger)
bigram_tagger_accuracy = bigram_tagger.evaluate(test_data)

print('Bigram tagger accuracy: ' + str(bigram_tagger_accuracy))



Bigram tagger accuracy: 0.8400679117147708


In [13]:

unigram_tagger = nltk.UnigramTagger(training_data, backoff=default_tagger)

bigram_tagger_accuracy = bigram_tagger.evaluate(test_data)
print('Bigram tagger accuracy: ' + str(bigram_tagger_accuracy))

Bigram tagger accuracy: 0.8400679117147708


In [77]:
import re
file_g = open('../../idn-tagged-corpus/Indonesian_Manually_Tagged_Corpus_ID.tsv')
g = file_g.read()
file_g.close()

print(repr(g[:100]))

p = re.compile(r'<kalimat id=\d+>((?:\n.+\t.+)+)\n</kalimat>')

raw_sentences = p.findall(g)


sentences2 = []
q = re.compile(r'(.+\t.+)')
for item in raw_sentences:
    tagged_sentence = []
    sep_sent = q.findall(item)
    for pair in sep_sent:
        sep_pair = pair.split('\t')
        tagged_sentence.append((sep_pair[0],sep_pair[1]))
    sentences2.append(tagged_sentence)
    
print(sentences2[:10])
    

'<kalimat id=1>\nKera\tNN\nuntuk\tSC\namankan\tVB\npesta olahraga\tNN\n</kalimat>\n<kalimat id=2>\nPemerintah\tNN'
[[('Kera', 'NN'), ('untuk', 'SC'), ('amankan', 'VB'), ('pesta olahraga', 'NN')], [('Pemerintah', 'NNP'), ('kota', 'NNP'), ('Delhi', 'NNP'), ('mengerahkan', 'VB'), ('monyet', 'NN'), ('untuk', 'SC'), ('mengusir', 'VB'), ('monyet-monyet', 'NN'), ('lain', 'JJ'), ('yang', 'SC'), ('berbadan', 'VB'), ('lebih', 'RB'), ('kecil', 'JJ'), ('dari', 'IN'), ('arena', 'NN'), ('Pesta Olahraga', 'NNP'), ('Persemakmuran', 'NNP'), ('.', 'Z')], [('Beberapa', 'CD'), ('laporan', 'NN'), ('menyebutkan', 'VB'), ('setidaknya', 'RB'), ('10', 'CD'), ('monyet', 'NN'), ('ditempatkan', 'VB'), ('di', 'IN'), ('luar', 'NN'), ('arena', 'NN'), ('lomba', 'NN'), ('dan', 'CC'), ('pertandingan', 'NN'), ('di', 'IN'), ('ibukota', 'NNP'), ('India', 'NNP'), ('.', 'Z')], [('Pemkot', 'NNP'), ('Delhi', 'NNP'), ('memiliki', 'VB'), ('28', 'CD'), ('monyet', 'NN'), ('dan', 'CC'), ('berencana', 'VB'), ('mendatangkan', 'VB'),

Tag Description Example
CC Coordinating conjunction dan, tetapi, atau
CD Cardinal number dua, juta, enam, 7916, sepertiga, 0,025, 0,525, banyak, kedua, ribuan, 2007, 25
OD Ordinal number ketiga, ke-4, pertama
DT Determiner / article Para, Sang, Si
FW Foreign word climate change, terms and conditions
IN Preposition dalam, dengan, di, ke, oleh, pada, untuk
JJ Adjective bersih, panjang, hitam, lama, jauh, marah, suram, nasional, bulat
MD Modal and auxiliary verb boleh, harus, sudah, mesti, perlu
NEG Negation tidak, belum, jangan
NN Noun monyet, bawah, sekarang, rupiah
NNP Proper noun
Boediono, Laut Jawa, Indonesia, India, Malaysia, Bank Mandiri, BBKP, Januari,
Senin, Idul Fitri, Piala Dunia, Liga Primer, Lord of the Rings: The Return of the
King
NND Classifier, partitive, and measurement noun orang, ton, helai, lembar
PR Demonstrative pronoun ini, itu, sini, situ
PRP Personal pronoun saya, kami, kita, kamu, kalian, dia, mereka
RB Adverb sangat, hanya, justru, niscaya, segera
RP Particle pun, -lah, -kah
SC Subordinating conjunction sejak, jika, seandainya, supaya, meski, seolah-olah, sebab, maka, tanpa, dengan,
bahwa, yang, lebih ... daripada ..., semoga
SYM Symbol IDR, +, %, @
UH Interjection brengsek, oh, ooh, aduh, ayo, mari, hai
VB Verb merancang, mengatur, pergi, bekerja, tertidur
WH Question siapa, apa, mana, kenapa, kapan, di mana, bagaimana, berapa
X Unknown statemen
Z Punctuation "...", ?, . 

ADJ, ADP, ADV, AUX, CCONJ, DET, NOUN, NUM, PART, PRON, PROPN, PUNCT, SYM, VERB, X

In [85]:
adjUPOS = [ x.lower() for sentence in training_data for (x,pos) in sentence if pos == 'ADJ']
adjXPOS = [ x.lower() for sentence in sentences2 for (x,pos) in sentence if pos == 'JJ']
print(len(set(adjUPOS)))
print(len(set(adjXPOS)))

set1_adjs = set(adjUPOS) - set(adjXPOS)
set2_adjs = set(adjXPOS) - set(adjUPOS)

print(len(set1_adjs))
print(len(set2_adjs))

print(set1_adjs)

1032
697
706
371
{'wangi', 'susah', 'ekslusif', 'sedih', 'putri', 'teliti', 'mencurigakan', 'memuaskan', 'persegi', 'seutuhnya', 'rimbun', 'bertahap', 'diam-diam', 'reveral', 'serumit', 'mundur', 'alternatif', 'abuan', 'berbobot', 'setim', 'jenis', 'barumu', 'ex', 'lembap', 'kurangnya', 'misionaris', 'curian', 'jengkel', 'seperjuangan', 'berisik', 'locally', 'tangkas', 'cilik', 'massif', 'mati', 'klayaban', 'lunas', 'universal', 'disekitarnya', 'wikinomics', 'eksotik', 'integral', 'banyaknya', 'manis', 'kebiruan', 'tercepat', 'lisan', 'sepsis', 'sekitarnya', 'emas', 'basah', 'murtad', 'bugar', 'numismatis', 'tersebar', 'molekul', 'sekaligus', 'normatif', 'positivis', 'visual', 'angkuh', 'ketakutan', 'esa', 'kecilnya', 'diam', 'spesialis', 'tertulis', 'genealogis', 'abadi', 'terburu', 'egois', 'piktoral', 'down', 'ekstrakurikuler', 'semangat', 'berkabung', 'berwibawa', 'temporal', 'a', 'modernis', 'kemerahan', 'analog', 'resminya', 'elastik', 'tiruan', 'tewas', 'harian', 'betina', 'leh'

In [81]:
print(len(training_data))
print(len(sentences2))



4477
9940
